In [1]:
import pandas as pd

In [4]:
path = '../data/msrvtt/MSRVTT-QA/train_qa.json'

In [5]:
df = pd.read_json(path)
df

,answer,category_id,id,question,video_id
0,man,9,0,who drives down the road in an audi?,0
1,show,9,1,what is a man doing?,0
2,drive,9,2,what is a man silently narrates his experience...,0
3,drive,9,3,what is a person doing?,0
4,tell,9,4,what is a person doing?,0
...,...,...,...,...,...
158576,two,7,158576,how many people are kissing each other?,6512
158577,man,7,158577,who kissing a woman?,6512
158578,two,7,158578,how many people are kissing?,6512
158579,man,7,158579,who is kissing a woman?,6512


## Create Answer Set

In [6]:
answer_freq = df['answer'].value_counts()
answer_freq.to_frame()

,answer
man,14801
talk,7493
woman,6599
person,5326
play,4905
...,...
pottasium,1
chastise,1
chopper,1
recording,1


### Take top 1000 words
https://github.com/xudejing/video-question-answering/blob/462f6e599fb02cee77c0b3d86901593efeb11c88/preprocess_msrvttqa.py#L64

In [7]:
top_1k = pd.DataFrame(answer_freq.iloc[:1000]).reset_index()
top_1k = top_1k['index']
top_1k

0            man
1           talk
2          woman
3         person
4           play
         ...    
995       number
996       engage
997    machinery
998       toilet
999       bucket
Name: index, Length: 1000, dtype: object

## Create Vocabulary
https://github.com/xudejing/video-question-answering/blob/462f6e599fb02cee77c0b3d86901593efeb11c88/preprocess_msrvttqa.py#L77

In [8]:
vocab = dict()

# Only take the questions in the top 1k
filtered_df = df[df['answer'].isin(top_1k)]

questions = filtered_df['question'].values

print(f'Number of original rows: {len(df)}')
print(f'Number of rows after filtering: {len(filtered_df)}')

Number of original rows: 158581
Number of rows after filtering: 144367


### Extract Tokens

In [9]:
for question in questions:
    words = question.rstrip('?').split()
    
    for word in words:
        if len(word) >= 2:
            vocab[word] = vocab.get(word, 0) + 1

vocab = pd.Series(vocab)
vocab.sort_values(ascending=False, inplace=True)
vocab

what           97959
is             89877
doing          49904
who            41192
in             26342
               ...  
pooped             1
pixie              1
bal                1
congressmen        1
radiation          1
Length: 16170, dtype: int64

In [10]:
# Taking the top 7999
vocab = pd.DataFrame(vocab.iloc[0:7999])
vocab.loc['<UNK>'] = [0]
vocab

,0
what,97959
is,89877
doing,49904
who,41192
in,26342
...,...
fondling,3
solubility,3
rotation,3
bills,3


In [12]:
vocab.to_csv('msrvtt_vocab', columns=[], header=False)